* Overview of Implementing Delete logic
* Update user detail HTML page for Deletion
* Review logic to delete data using flask shell
* Add delete user logic to the application
* Review delete user logic
* Run application and validate
* Exercise and Solution

* Overview of Implementing Delete logic
  * `DELETE` is one of the REST APIs
  * HTML Forms typically supports `GET` and `POST`
  * We will implement the `DELETE` functionality using `POST`

* Update user detail HTML page for Deletion

Update `templates/user_detail.html`

```html
{%- extends "base.html" %}

{% block content %}
<form method="POST" action="{{ url_for('user', id=user.id) }}">
    <h3>User Details</h3>
    <div class="container">
        <label for="first_name"><b>First Name</b></label>
        {{ user.first_name }}
        <br><br>
        <label for="last_name"><b>Last Name</b></label>
        {{ user.last_name }}
        <br><br>
        <label for="username"><b>Username</b></label>
        {{ user.username }}
        <br><br>
        <label for="email"><b>Email</b></label>
        {{ user.email }}
        <br><br>

        <button type="submit" name="action" value="edit">Edit</button>
        <button type="submit" name="action" value="delete">Delete</button>
    </div>
</form>
{%- endblock %}
```

* Review logic to delete data using flask shell

Launch flask shell using `python -m flask shell` and run the below code snippets.
```python
from models.user import User
User.query.all() # Use the first id
u = User.query.get(1)
db.session.delete(u)
db.session.commit() # Updates the data in the database table

User.query.all()
# The result will not contain the record deleted
```

Use pgAdmin and run the following query to confirm.
```sql
SELECT * FROM users WHERE id = 1;
```

* Add delete user logic to the application

Update `user` function in `app.py` with delete functionality.

```python
@app.route('/user', methods=['GET', 'POST'])
def user():
    if request.method == 'GET':
        id = request.args.get('id')
        if id:
            user = User.query.get(id)
            return render_template('user_detail.html', user=user)
        else:
            return render_template('user_form.html', user=None)
    elif request.method == 'POST':
        id = request.args.get('id')
        if id:
            user = User.query.get(id)
            form_action = request.form['action']
            if form_action == 'edit':
                return render_template('user_form.html', user=user)
            elif form_action == 'delete':
                db.session.delete(user)
                db.session.commit()
                return redirect(url_for('users'))
        else:
            id = request.form['id']
            first_name = request.form['first_name']
            last_name = request.form['last_name']
            username = request.form['username']
            email = request.form['email']
            if id:
                user = User.query.get(id)
                user.first_name = first_name
                user.last_name = last_name
                user.username = username
                user.email = email
            else:
                user = User(
                    first_name=first_name, 
                    last_name=last_name, 
                    username=username,
                    email=email
                )
                db.session.add(user)
            db.session.commit()
            return redirect(url_for('users'))
```

* Review delete user logic
  * Changes to user detail page
  * Changes to application to delete the data.

* Run application and validate

Run the application using Flask.
```shell
python -m flask run
```

Here are the steps to validate the application.
* Visit Users Dashboard http://localhost:5000/users
* Visit User Details page for 1 - http://localhost:5000/users?id=1
* Click on **Delete** button.
* The page will be redirected to users page. Make sure to review and confirm if the user is deleted from the database or not.
* You can also connect to pgAdmin and run `SELECT` to confirm if the user is deleted or not.

* Exercise - Develop functionality to delete course details to the table.
  * Update course detail html page (course_form.html) to form to update existing course details. 
  * Update function `course` for `/course` end point with `POST` to delete data in the table.
  * Run and Validate the application. Go to the end point `/course` with `?course_id=1` parameter and perform below steps.
    * Make sure to click on **Delete** button.
    * Make sure that the page is redirected to **courses** dashboard.
    * The course deleted should not be there in the dashboard.